# Mantenimiento
- Buscar cosas descripciones de trabajos efectuados por MEL en mantenimiento
- Buscar detenciones

In [1]:
### CORRERLO INICIALENTE ###
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

import pandas as pd
import polars as pl
import dagster as dg
from kdags.resources.dplyr import *
from kdags.resources.tidyr import *
from io import BytesIO
import pdfplumber

from datetime import datetime
import re
import os
from kdags.definitions import *
from kdags.readr import *

context = dg.build_asset_context()
### IGNORAR ###

# Historial de órdenes de trabajo



In [2]:
# CORRRER
work_order_history_df = Readr.Maintenance.read_work_order_history()

## Busqueda específica

In [6]:
work_order_history_df["equipment_name"].unique()

array(['TK320', 'TK322', 'TK396', 'TK397', 'TK398', 'TK399', 'TK847',
       'TK848', 'TK849', 'TK850', 'TK851', 'TK852', 'TK854', 'TK855',
       'TK856', 'TK857', 'TK858', 'TK859', 'TK860', 'TK861', 'TK862',
       'TK863', 'TK864', 'TK865', 'TK866', 'TK867', 'TK868', 'TK869',
       'TK871', 'TK872', 'TK873', 'TK874', 'TK875', 'TK876', 'TK877',
       'TK878', 'TK879', 'TK880'], dtype=object)

In [15]:
# INICIO MODIFICACION
start_date = "2024-01-30"
end_date = "2024-07-30"
equipment_name = "TK322"
# FIN DE MODIFICACION
df = work_order_history_df.copy()
df = (
    df.loc[
        (df["equipment_name"] == equipment_name)
        & (df["start_date"] >= datetime.strptime(start_date, "%Y-%m-%d"))
        & (df["end_date"] <= datetime.strptime(end_date, "%Y-%m-%d"))
    ]
    .sort_values("end_date", ascending=False)
    .reset_index(drop=True)
)
df = df.assign(text=df["description"] + " " + df["task_text"])
df[["start_date", "end_date", "description", "task_text"]]

,start_date,end_date,description,task_text
0,2024-07-23,2024-07-24,6000H Mec Proc Carga Susp TK322,27.09.2023 18:52:31 UTC (FIGULAU)\n\n **RPLPRI...
1,2024-07-23,2024-07-23,IS; Cbiar línea trasera manif direcc,02.07.2024 09:11:53 UTC (LOPEWEN)\n En inspecc...
2,2024-07-23,2024-07-23,IS; Cambiar botapiedra derecho,02.07.2024 09:49:57 UTC (LOPEWEN)\n En inspecc...
3,2024-07-23,2024-07-23,Reparar soporte silenciador,08.03.2024 07:21:47 UTC (BOSSWREMOTE)\n Brief ...
4,2024-07-23,2024-07-23,IS; Retiro perno cortado soporte scoop,02.05.2024 21:00:36 UTC (HERRVG)\n En inspecci...
...,...,...,...,...
255,2024-02-05,2024-02-05,12000H Sol CCE Cert Punt Anclj TK322,05.02.2024 21:55:47 UTC (GODOCAMI) Tel. +56 9 ...
256,2024-02-05,2024-02-05,28D Mec Serv Sist Incen TK322,05.02.2024 21:16:19 UTC (CORTANG)\n Se realiza...
257,2024-02-05,2024-02-05,28D Serv CCE Insp Tcnolg Paso 2 TK322,05.02.2024 23:01:35 UTC (ARAYDIEGO)\n se reali...
258,2024-02-01,2024-02-01,Falla TMC2 (targeta de control MT2),01.02.2024 19:41:16 UTC (VALEPI)\n Brief descr...


In [16]:
df["task_text"].to_list()

['27.09.2023 18:52:31 UTC (FIGULAU)\n\n **RPLPRI**\n Aplazado por priorización de otros trabajos\n 12.10.2023 00:18:23 UTC (FIGULAU)\n **RPLPRI**\n Aplazado por priorización de otros trabajos\n Barra estabilizadora del mismo equipo\n 05.11.2023 00:32:42 UTC (FIGULAU)\n **RPLPRI**\n Aplazado por priorización de otros trabajos\n 04.12.2023 02:04:16 UTC (FIGULAU)\n **RPLPRI**\n Aplazado por priorización de otros trabajos\n 28.02.2024 21:50:15 UTC (ROJAGE4) Phone +56990934989\n **RPLPRI**\n Aplazado por priorización de otros trabajos\n 30.04.2024 20:06:56 UTC (HIGUKA)\n Se desplaza OT por cambio Modulo MTU TK861 en W22.\n 25.06.2024 15:11:50 UTC (HIGUKA)\n Se desplaza OT por capacidad y priorización de otros trabajos.\n 23.07.2024 08:33:59 UTC (RODRPD)\n se realiza procedimiento de carga suspensiones segun procedimiento\n quedando tarea finalizada.',
 '02.07.2024 09:11:53 UTC (LOPEWEN)\n En inspección se detectó leve fuga por el prensado de manguera trasera\n del manifold de dirección. Se 

In [5]:
# BUSCADOR ESPECIAL
i = 30
print(df.iloc[i]["description"])
print(df.iloc[i]["task_text"])

TKD858 SMARTCAP cambio de banda
10.05.2024 17:07:54 UTC (BUSTJA)

 TKD858 SMARTCAP cambio de banda
 Equipo a Reparar: TKD-858
 Tecnología: Smartcap bandalife
 Nombre del Operador: Lea Astudillo
 Fecha de la actividad: 10/05/2024
 Hora de detención de equipo: 11:58
 Hora inicio actividad: 12:28
 Hora término de actividad: 12:40
 Falla informada al CIO: Banda life no registra niveles
 Falla encontrada: Banda life en mal estado
 Solución de la falla: se ingresa a la unidad chequeando banda life de
 operadora en mal estado, se procede a realizar cambio de banda y
 entregando con todos sus accesorios y también asignando nueva banda y
 haciendo pruebas de niveles y comunicación junto a despacho smartcap
 quedando operativo
 Lugar de atención: estacionamiento dique a
 Estado final: operativo

 Detalles de bandas:
 Banda entrante: B8DA
 Banda saliente: 9DE0




 02 tecnicos, 20 minutos


In [6]:
print("Descripciones de mantenimiento:")
df["task_text"].to_list()

Descripciones de mantenimiento:


['14.06.2024 07:17:16 UTC (MIRAVI)\n Brief description of what occurred (e.g. basic sequence of events and\n impacts, avoiding the use of personnel names) ...\n fuga de aceite de direccion\n ----------------------------------------------------------------\n Immediate action taken ...\n crea aviso, evaluar fuga de aceite de direccion\n ----------------------------------------------------------------\n Define work requirements; including known materials, specialist labour,\n special tools that are required ...\n 2 mcanicos2 horas\n ----------------------------------------------------------------\n Define any equipment constraints ...\n ----------------------------------------------------------------\n Define any other information to support work requirements ...\n sin task list\n ----------------------------------------------------------------\n 14.06.2024 10:33:17 UTC (MIRAVI)\n Equipo presenta fuga de aceitede direccion por valvula repartidora de\n cilindros de direccion, sello cortado